Assistant

In [1]:
import ollama

def procesar_tarea_llm(tarea: str):
    prompt = f"""
    Tengo la siguiente tarea: "{tarea}".

    Por favor:
    1. Clasifícala como 'alta', 'media' o 'baja' prioridad.
    2. Justifica brevemente tu elección.
    3. Da 2 recomendaciones útiles para completarla de forma efectiva.
    """

    respuesta = ollama.chat(
        model="mistral",
        messages=[{"role": "user", "content": prompt}]
    )

    return respuesta["message"]["content"]

# Bucle de ejemplo
while True:
    tarea = input("\n📥 Ingresa una tarea (o escribe 'salir'): ")
    if tarea.lower() == "salir":
        break

    respuesta = procesar_tarea_llm(tarea)
    print("\n🤖 Asistente inteligente dice:\n")
    print(respuesta)



🤖 Asistente inteligente dice:

 1. Alta Prioridad: Visitar a una novia siempre es importante en un relación, especialmente en el caso de una pareja enamorada. Es una actividad fundamental que ayuda a fortalecer la conexión emocional entre las personas y mantener la calidez en una relación sana y equilibrada.
2. Visitar a tu novia siempre es importante, pero dependiendo de la situación personal actual, la prioridad puede variar. En el caso de que hayas estado separados por mucho tiempo o que tengas problemas que discutir, la prioridad sería más alta. Sin embargo, si tu relación está en un buen momento y se encuentran juntos regularmente, la prioridad podría ser más baja.
3. Recomendaciones útiles para completarla de forma efectiva:
    - Prepara lo mejor que puedas el espacio donde vas a pasar tiempo con tu novia. Esto incluye limpiar y arreglar su casa o espacio común, encender una vela, preparar música agradable o incluso planear actividades divertidas para aumentar la calidez del mo

 Embeddings para encontrar tareas similares antes de preguntarle al LLM

In [2]:
from sentence_transformers import SentenceTransformer, util
import ollama

# 🔹 Cargamos modelo de embeddings
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# 🔸 Base de conocimiento simple (tareas + embeddings + respuestas LLM)
tareas_previas = []

# 🔹 Buscar tarea similar
def buscar_similar(tarea_nueva, umbral=0.8):
    vector_nuevo = embedder.encode(tarea_nueva, convert_to_tensor=True)

    for tarea, vector, respuesta in tareas_previas:
        similitud = util.cos_sim(vector_nuevo, vector)[0].item()
        if similitud >= umbral:
            return respuesta, similitud

    return None, None

# 🔸 Llamar a Ollama
def consultar_llm(tarea):
    prompt = f"""
    Tengo la siguiente tarea: "{tarea}".

    Por favor:
    1. Clasifícala como 'alta', 'media' o 'baja' prioridad.
    2. Justifica brevemente tu elección.
    3. Da 2 recomendaciones útiles para completarla de forma efectiva.
    """
    respuesta = ollama.chat(
        model="mistral",
        messages=[{"role": "user", "content": prompt}]
    )
    return respuesta["message"]["content"]

# 🔁 Bucle de interacción
while True:
    tarea_usuario = input("\n📥 Ingresa una tarea (o escribe 'salir'): ")
    if tarea_usuario.lower() == "salir":
        break

    respuesta_similar, similitud = buscar_similar(tarea_usuario)

    if respuesta_similar:
        print(f"\n🔁 Esta tarea es muy similar a otra previa (similitud: {similitud:.2f}). Reutilizando respuesta:")
        print(respuesta_similar)
    else:
        respuesta = consultar_llm(tarea_usuario)
        print("\n🤖 Asistente inteligente dice:\n")
        print(respuesta)

        # Guardamos
        embedding_nuevo = embedder.encode(tarea_usuario, convert_to_tensor=True)
        tareas_previas.append((tarea_usuario, embedding_nuevo, respuesta))



🤖 Asistente inteligente dice:

 1. Prioridad alta: Ambas áreas (baño y cocina) son espacios habitualmente usados en la vida diaria, por lo que su limpieza es importante para la salud y bienestar.
2. La limpieza de un baño y una cocina es fundamental para evitar el crecimiento de bacterias patógenas y mantener el ambiente limpio y saludable.
3. Recomendaciones útiles:
   - Para limpiar efectivamente, utiliza productos limpiadores apropiados en cada superficie. Por ejemplo, para el baño, es importante usar productos desinfectantes en los baños y duchas, y productos abrasivos ligeramente en las zonas de acero inoxidable y madera. En la cocina, utiliza productos apropiados para la limpieza de superficies comestibles, como aceite de limón o un desinfectante aprobado para cocinas.
   - Ordena y organiza los artículos de uso cotidiano en sus lugares correspondientes. Esto ayuda a mantener el ambiente más limpio y evita que se acumule suciedad alrededor de ellos. En el baño, puede utilizar ce

KeyboardInterrupt: 